# Data Wrangling - Chicago Crime

## Set up the environment

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import math
import googlemaps
from collections import Counter

In [2]:
import os
os.getcwd()

'C:\\Users\\aroge\\Documents\\GitHub\\Springboard\\Chicago_Crime'

## Import the file and inspect its basic characteristics

In [3]:
df = pd.read_csv('crimes_2019.csv', index_col='ID') 

In [4]:
df.shape

(259210, 21)

In [5]:
df.head()

,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
ID,,,,,,,,,,,,,,,,,,,,,
11734802,JC301648,6/11/2019 6:00,033XX W FILLMORE ST,1812,NARCOTICS,POSSESS - CANNABIS MORE THAN 30 GRAMS,POLICE FACILITY / VEHICLE PARKING LOT,False,False,1134,...,24.0,29,18,NaN,NaN,2019,6/18/2020 15:48,NaN,NaN,NaN
12080437,JD265831,11/1/2019 5:00,063XX S CALIFORNIA AVE,486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,825,...,17.0,66,08B,NaN,NaN,2019,6/18/2020 15:48,NaN,NaN,NaN
11721309,JC288956,6/2/2019 12:22,044XX W WASHINGTON BLVD,2017,NARCOTICS,MANUFACTURE / DELIVER - CRACK,VEHICLE NON-COMMERCIAL,False,False,1113,...,28.0,26,18,NaN,NaN,2019,6/18/2020 15:48,NaN,NaN,NaN
12078686,JD264973,6/15/2019 0:01,014XX W GEORGE ST,530,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,ALLEY,False,False,1932,...,32.0,6,04A,1166295.0,1919354.0,2019,6/18/2020 15:46,41.934286,-87.664279,"(41.93428599, -87.66427875)"
11725655,JC286220,5/31/2019 12:05,079XX S DR MARTIN LUTHER KING JR DR,2014,NARCOTICS,MANUFACTURE / DELIVER - HEROIN (WHITE),SIDEWALK,False,False,623,...,6.0,44,18,NaN,NaN,2019,6/18/2020 15:48,NaN,NaN,NaN


## Check the columns for null values

In [6]:
df.isnull().sum()

Case Number                0
Date                       0
Block                      0
IUCR                       0
Primary Type               0
Description                0
Location Description    1066
Arrest                     0
Domestic                   0
Beat                       0
District                   0
Ward                      15
Community Area             0
FBI Code                   0
X Coordinate            1177
Y Coordinate            1177
Year                       0
Updated On                 0
Latitude                1177
Longitude               1177
Location                1177
dtype: int64

Nulls for the location-based columns is our biggest problem since the purpose of this project is to study crime based on other factors at the same location. This will be addressed below.

## Explore the contents of the data, column by column

In [7]:
df['Block'].value_counts()

001XX N STATE ST         1087
008XX N MICHIGAN AVE      472
0000X W TERMINAL ST       427
0000X S STATE ST          411
0000X N STATE ST          363
                         ... 
033XX N OKETO AVE           1
099XX S PRAIRIE AVE         1
057XX S CLAREMONT AVE       1
038XX W ROSEMONT AVE        1
065XX N WAUKESHA AVE        1
Name: Block, Length: 27777, dtype: int64

In [8]:
df['IUCR'].value_counts().sort_index()

031A    2640
031B      45
033A     239
033B       6
041A    1606
        ... 
928        1
930      593
935       26
937        1
938        1
Name: IUCR, Length: 318, dtype: int64

In [9]:
df['Primary Type'].value_counts()

THEFT                                62387
BATTERY                              49487
CRIMINAL DAMAGE                      26671
ASSAULT                              20608
DECEPTIVE PRACTICE                   18330
OTHER OFFENSE                        16696
NARCOTICS                            14238
BURGLARY                              9627
MOTOR VEHICLE THEFT                   8978
ROBBERY                               7989
CRIMINAL TRESPASS                     6817
WEAPONS VIOLATION                     6325
OFFENSE INVOLVING CHILDREN            2293
INTERFERENCE WITH PUBLIC OFFICER      1546
PUBLIC PEACE VIOLATION                1520
SEX OFFENSE                           1297
CRIM SEXUAL ASSAULT                   1244
PROSTITUTION                           680
HOMICIDE                               503
ARSON                                  378
CRIMINAL SEXUAL ASSAULT                356
LIQUOR LAW VIOLATION                   231
STALKING                               223
CONCEALED C

In [10]:
df['Location Description'].value_counts()

STREET                                       56450
RESIDENCE                                    42878
APARTMENT                                    34638
SIDEWALK                                     20181
OTHER                                        10563
                                             ...  
COLLEGE / UNIVERSITY - GROUNDS                   1
SPORTS ARENA / STADIUM                           1
FACTORY / MANUFACTURING BUILDING                 1
CTA PARKING LOT / GARAGE / OTHER PROPERTY        1
CHA GROUNDS                                      1
Name: Location Description, Length: 155, dtype: int64

In [11]:
df['Arrest'].value_counts().sort_index()

False    204169
True      55041
Name: Arrest, dtype: int64

In [12]:
df['Domestic'].value_counts().sort_index()

False    216140
True      43070
Name: Domestic, dtype: int64

In [13]:
df['Beat'].value_counts().sort_index()

111     2496
112     2165
113     1049
114     1190
121      786
        ... 
2531     686
2532    1018
2533    1597
2534    1193
2535     837
Name: Beat, Length: 274, dtype: int64

In [14]:
df['District'].value_counts().sort_index()

1     15198
2     11182
3     12433
4     13975
5     11307
6     16778
7     13600
8     15596
9     10915
10    12431
11    18301
12    13123
14     9239
15     9951
16     8199
17     6719
18    15124
19    11770
20     4345
22     8207
24     7956
25    12854
31        7
Name: District, dtype: int64

In [15]:
df['Ward'].value_counts().sort_index()

1.0      4799
2.0      5254
3.0      7348
4.0      6923
5.0      5892
6.0      9700
7.0      7506
8.0      8119
9.0      7849
10.0     3721
11.0     3086
12.0     3206
13.0     2245
14.0     2929
15.0     3786
16.0     7877
17.0     8362
18.0     3773
19.0     2005
20.0     8005
21.0     8276
22.0     3320
23.0     2450
24.0    11024
25.0     4801
26.0     3810
27.0    12390
28.0    12860
29.0     5912
30.0     2665
31.0     2892
32.0     3257
33.0     2376
34.0     6666
35.0     2813
36.0     2816
37.0     7753
38.0     1818
39.0     2204
40.0     2729
41.0     2828
42.0    16604
43.0     3065
44.0     4217
45.0     2493
46.0     3233
47.0     2353
48.0     2581
49.0     3997
50.0     2607
Name: Ward, dtype: int64

In [16]:
df['Community Area'].value_counts().sort_index()

1     4037
2     3441
3     3339
4     1793
5     1270
      ... 
73    3207
74     466
75    1868
76    1623
77    2493
Name: Community Area, Length: 77, dtype: int64

In [17]:
df['FBI Code'].value_counts().sort_index()

01A      495
01B        8
04A     5840
04B     7860
08A    18358
08B    42363
10      1560
11     16661
12        31
13        43
14     26671
15      6545
16       680
17      1864
18     14243
19       143
2       1768
20       804
22       231
24      3033
26     20650
3       7989
5       9627
6      62387
7       8978
9        378
Name: FBI Code, dtype: int64

## Address the null values

For the rows that have null location data, create a dictionary of replacement values 

Are there other rows in our data that have the same "Block" value and have valid location data? Take the mean of those values for our dictionary.

In [18]:
# Create a DataFrame of just rows with null location data
df_nulls = df[df.Latitude.isnull()]
print("Before removing duplicates,", df_nulls.shape[0], "rows have NULL location data.")

# Remove duplicate rows in df_nulls
df_nulls = df_nulls.drop_duplicates(subset='Block', keep="first")
print("After removing duplicates,", df_nulls.shape[0], "rows have NULL location data.")

Before removing duplicates, 1177 rows have NULL location data.
After removing duplicates, 982 rows have NULL location data.


In [19]:
# Create latitude and longitude dictionaries for rows with missing location data. The key is the Block column
# Every row with a null location has a value for Block. Take the mean of the other rows with the same Block. 
#
# THIS TAKES SEVERAL SECONDS TO RUN

block_lat = {}
block_long = {}
for block in df_nulls.Block:
    block_lat[block] = df[df.Block == block].Latitude.mean()
    block_long[block] = df[df.Block == block].Longitude.mean()

In [20]:
# Now that we have the dictionaries, update the original dataframe

# Gather the indexes of the rows that have NULL location data
null_loc_indexes = df[df.Latitude.isnull()].index

# Update the location data with the dictionary values
#
# THIS TAKES A FEW SECONDS TO RUN

for i in null_loc_indexes:
    df.loc[df.index == i, 'Latitude'] = block_lat[df.loc[i].Block]
    df.loc[df.index == i, 'Longitude'] = block_long[df.loc[i].Block]
    df.loc[df.index == i, 'Location'] = str(tuple([block_lat[df.loc[i].Block], block_long[df.loc[i].Block]]))

In [21]:
# How many rows still have NULL location data?
print("Still have", df[df.Latitude.isnull()].shape[0], "rows with NULL location data.")

Still have 193 rows with NULL location data.


Given that less than 0.1% of rows have missing location data, we will delete these rows.

In [22]:
# Print the number of rows in the original dataframe
print("Number of rows in original dataframe =", df.shape[0])

# Create a new dataframe without the rows with NULL location data
null_locations = df[df.Latitude.isnull()].index
crimes = df.drop(null_locations)

# Print the number of rows in the new dataframe
print("Number of rows in cleaned dataframe =", crimes.shape[0])

Number of rows in original dataframe = 259210
Number of rows in cleaned dataframe = 259017


We see the number of rows drops equals the number of rows that had NULL location data.